In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 🌈 Step 1: Generate dummy images
def create_color_image(color, size=(224, 224)):
    img = np.ones((size[0], size[1], 3), dtype=np.uint8)
    img[:] = color
    return img

# Create dataset directories
os.makedirs("dummy_data/acceptable", exist_ok=True)  # Blue-ish
os.makedirs("dummy_data/unacceptable", exist_ok=True)  # Pink/White

# Generate blue to pink to white images
for i in range(100):
    # Acceptable (blue shades)
    blue = (255 - i*2, 0, i*2)  # BGR
    img = create_color_image(blue)
    cv2.imwrite(f"dummy_data/acceptable/blue_{i}.jpg", img)

    # Unacceptable (pink to white)
    pink = (255, 192 + i//2, 203 + i//3)
    img = create_color_image(pink)
    cv2.imwrite(f"dummy_data/unacceptable/pink_{i}.jpg", img)

# 🌱 Step 2: Load and preprocess the images
def load_images_from_folder(folder):
    data = []
    labels = []
    for label, subfolder in enumerate(["acceptable", "unacceptable"]):
        path = os.path.join(folder, subfolder)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            img = cv2.resize(img, (64, 64))
            data.append(img / 255.0)
            labels.append(label)
    return np.array(data), np.array(labels)

X, y = load_images_from_folder("dummy_data")
y_cat = to_categorical(y)

# 📊 Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# 🧠 Step 4: Simple CNN
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 🏋️ Step 5: Train
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# 📈 Step 6: Plot results
plt.plot(history.history['accuracy'], label="train_acc")
plt.plot(history.history['val_accuracy'], label="val_acc")
plt.title("Dummy Model Accuracy")
plt.legend()
plt.show()
